In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website to scrape
url = "https://pravda.com.ua"

# Send a GET request to the website
response = requests.get(url)

# Parse the content of the request with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the div with the class "war_block"
war_section = soup.find('div', class_='war_block')

# Extract the information on casualties
casualties = []
if war_section:
    items = war_section.find_all('div', class_='war_item')
    for item in items:
        value = item.find('div', class_='war_num').text.split('+')[0].strip('~')
        description = item.find('div', class_='war_desc').text.strip()
        casualties.append({"Category": description, "Value": value})

# Create a DataFrame from the extracted information
df = pd.DataFrame(casualties)

# Display the DataFrame
print(df)


                                 Category   Value
0                      В особовому складі  534360
1                                 Літаків     359
2                            Гелікоптерів     326
3                                  Танків    8019
4                                     ББМ   15398
5                   Артилерійських систем   14195
6                              Засоби ППО     863
7                                    РСЗВ    1108
8   Автомобільної техніки і цистерн з ПММ   19248
9                       Кораблі та катери      28
10               Оперативно-тактичні БПЛА   11355


In [3]:
df

""


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# URL of the website to scrape
url = "https://pravda.com.ua"

# Send a GET request to the website
response = requests.get(url)

# Parse the content of the request with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the div with the class "war_block"
war_section = soup.find('div', class_='war_block')

# Extract the information on casualties
casualties = {}
if war_section:
    items = war_section.find_all('div', class_='war_item')
    for item in items:
        value = item.find('div', class_='war_num').text.split('+')[0].strip('~')
        description = item.find('div', class_='war_desc').text.strip()
        casualties[description] = value

# Get the current date
current_date = datetime.now().strftime('%Y-%m-%d')

# Create a DataFrame from the extracted information with the date as the row
df = pd.DataFrame(casualties, index=[current_date])

# Display the DataFrame
print(df)


           В особовому складі Літаків Гелікоптерів Танків    ББМ  \
2024-06-23             534360     359          326   8019  15398   

           Артилерійських систем Засоби ППО  РСЗВ  \
2024-06-23                 14195        863  1108   

           Автомобільної техніки і цистерн з ПММ Кораблі та катери  \
2024-06-23                                 19248                28   

           Оперативно-тактичні БПЛА  
2024-06-23                    11355  


In [36]:
from PIL import Image
import pytesseract

# Load the image
image_path = "../input/test_image.jpg"
image = Image.open(image_path)

# Perform OCR on the image
extracted_text = pytesseract.image_to_string(image, lang='ukr+eng')

extracted_text

'Протягом 24.02.22 - 23.06.2024\n\nОРІЄНТОВНІ ВТРАТИ\nПРОТИВНИКА СКЛАЛИ\n\n534360 4270 359 326\n\nособового складу літаків гелікоптерів\npersonnel aircraft helicopters\n8019 «ic 11355 «50\n\nтанків БПЛА оперативно-тактичного рівня\ntanks UAV operational-tactical level\n15398 5 2321 «12\n\nбойових броньованих машин крилаті ракети\n\narmoured personnel vehicle cruise missiles\n\n14195 5: 28 1\nартилерійських систем кораблі/катери підводні човни\nartillery systems warships/boats submarines\n1108 «2 19248 «24\n\nPC3B автомобільна техніка Ta цистерни 3 ПММ\nMLRS vehicles & fuel tanks\n\n863 +2 2377 +2\n\nзасоби ППО спеціальна техніка\n\nanti-aircraft warfare systems special equipment\n\nШ Генеральний Штаб Збройних Сил України\n'

In [47]:
import re
import pandas as pd
from PIL import Image
import pytesseract
import cv2

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    return Image.fromarray(thresh)

def extract_text_from_image(image):
    return pytesseract.image_to_string(image, lang='ukr+eng')

# Main script
image_path = "../input/test_image.jpg"  # Update with the correct path to your image file

# Preprocess the image
preprocessed_image = preprocess_image(image_path)

# Extract text from the preprocessed image
extracted_text = extract_text_from_image(preprocessed_image)

# Save the extracted text to a file
with open("extracted_text.txt", "w", encoding="utf-8") as file:
    file.write(extracted_text)

print("Extracted text saved to extracted_text.txt")


Extracted text saved to extracted_text.txt


In [48]:
import re
import pandas as pd

def match_category(text, category, pattern):
    match = re.search(pattern, text)
    if match:
        try:           total = int(match.group(1).replace(',', ''))
            change = int(match.group(2).replace(',', '')) if match.group(2) else 0
            return {category: {"Total": total, "Change": change}}
        except IndexError:
            return {category: {"Total": 0, "Change": 0}}
    return {category: {"Total": 0, "Change": 0}}

def parse_extracted_text(text):
    patterns = {
        "Personnel": r'~(\d+)\s*[::]+(\d+)',
        "Tanks": r'(\d+)\s*\+(\d+)\nтанків\ntanks',
        "Armoured Personnel Vehicle": r'(\d+)\s*[«\+](\d+)\nбойових броньованих машин\narmoured personnel vehicle',
        "Artillery Systems": r'(\d+)\s*\+(\d+)\nартилерійських систем\nartillery systems',
        "MLRS": r'(\d+)\s*\+(\d+)\nPC3B\nMLRS',
        "Aircraft": r'(\d+)\s*\d*\nлітаків\naircraft',
        "Helicopters": r'(\d+)\s*\d*\nгелікоптерів\nhelicopters',
        "UAV Operational-Tactical Level": r'(\d+)\s*\+(\d+)\nБПЛА оператизно-тактичного рівня\nUAV operational-tactical level',
        "Cruise Missiles": r'(\d+)\s*\+(\d+)\nкрилаті ракети\ncruise missiles',
        "Warships/Boats": r'(\d+)\s*\d*\nкораблі/катери\nwarships/boats',
        "Submarines": r'(\d+)\s*\d*\нпідводні човни\nsubmarines',
        "Vehicles & Fuel Tanks": r'(\d+)\s*\+(\d+)\навтомобільна техніка та цистерни з ПММ\nvehicles & fuel tanks',
        "Anti-Aircraft Warfare Systems": r'(\д+)\s*\+\d+\нзасоби ППО\nanti-aircraft warfare systems',
        "Special Equipment": r'(\д+)\s*\+(\д+)\нспеціальна техніка\nspecial equipment'
    }
    
    data = {}
    for category, pattern in patterns.items():
        matched_data = match_category(text, category, pattern)
        if matched_data[category]["Total"] == 0:
            print(f"Pattern did not match for category: {category}")
        data.update(matched_data)
        
    return data

# Read the extracted text from the file
with open("extracted_text.txt", "r", encoding="utf-8") as file:
    extracted_text = file.read()

print("Extracted Text:\n", extracted_text)  # Print to check extracted text

# Parse the extracted text to structured data
parsed_data = parse_extracted_text(extracted_text)

# Convert dictionary to DataFrame
df = pd.DataFrame(parsed_data).transpose()

# Display the DataFrame
print("Parsed DataFrame:\n", df)
print("Tanks data:", df.loc['Tanks'] if 'Tanks' in df.index else "Tanks data not found")


Extracted Text:
 Протягом 24.02.22 - 23.06.2024

ОРІЄНТОВНІ ВТРАТИ
ПРОТИВНИКА СКЛАЛИ

~534360 ::270

особового складу
personnel

8019 +10

танків
tanks

15398 «15

бойових броньованих машин
armoured personnel vehicle

14195 +6:

артилерійських систем
artillery systems

PC3B
MLRS

863 +2

засоби ППО
anti-aircraft warfare systems

359 326

літаків гелікоптерів
aircraft helicopters

11355 +50

БПЛА оператизно-тактичного рівня
UAV operational-tactical level

2321 +13

крилаті ракети
cruise missiles

28 1

кораблі/катери підводні човни
warships/boats submarines

19248 «a4

автомобільна техніка та цистерни 3 ПММ
vehicles & fuel tanks

2377 +8

eneuianbya техніка
special equipment

Ш Генеральний Штаб Збройних Сил України

Pattern did not match for category: Tanks
Pattern did not match for category: Armoured Personnel Vehicle
Pattern did not match for category: Artillery Systems
Pattern did not match for category: MLRS
Pattern did not match for category: Aircraft
Pattern did not match for cate